# [Std 15] - Find EoM for and solve it for Denver to London.

In [1]:
########
## Env Setup
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

from sage.all import *

import numpy as np
np.random.seed(1_618_033)

################

Parallelism().set(nproc=8)
version()


'SageMath version 10.5, Release Date: 2024-12-04'

---

In [2]:
########
## Geometry & Discretization
##    a.k.a. "Pick a Coordinate System"
########

var("R")

# create a surface manifold of the sphere, embedded in Euclidean R^3 
S2 = manifolds.Sphere(
    2,          # dimension
    radius=R,   # symbolic variable radius
    latex_name="{\mathcal{S}^2}_R"
)

# define a coordinate system mapping, in the dimension of the surface manifold
spherical.<θ,φ> = S2.chart(r'θ:[0,2*pi):periodic:\theta φ:(-pi/2,pi/2):\phi')
S2.set_default_chart(spherical)

#g = S2.metric()
#g[0,0], g[1,1] = R^2, R^2 * sin(θ)

print(spherical)
spherical.coord_range()


Chart (S^2_R, (θ, φ))


θ: [0, 2*pi] (periodic); φ: (-1/2*pi, 1/2*pi)

In this case, we have defined a mapping of the basis vectors on the surface of the 2-Sphere (a _Chart_); with azimuthal angle in $\theta$, and inclination angle $\phi$ from the equator.  That is, the equator will have inclination angle $\phi=0$, and is periodic in $\theta \in [0, 2\pi)$.  We will also choose the convention that the "North Pole" of $\mathcal{S}^2$ is located at $\phi = -\frac{\pi}{2}$, to preserve the context of "negative sign is counter-clockwise".

Since we have defined a symbolic variable radius $R$, points are effectively evaluated on the unit 2-Sphere ($R=1$).  We will instantiate a value for $R$ later; for our purposes of calculating path length $\mathrm{d}s$ here, $R \approx 6371\ [\mathrm{km}]$. 

Now, we need to define coordinate mappings, from the latitudinal and longitudinal geographic coordinate system, and the Cartesian coordinate system in Euclidean space, to the coordinate mapping we have defined on the 2-Sphere manifold surface.  Accordingly, latitudes are recorded with $+90^\circ$ at the "North Pole", $-90^\circ$ at the "South Pole"; while longitudes are recorded with negative sign West (clockwise along equator w.r.t. Prime Meridian from Observer at North Pole), and positive sign for East (CCW on Equator w.r.t. PM from NP observer). 

In geographic coordinate systems, longitude is denoted with $\lambda$.  While latitudinal inclination from the equator is _also_ normally denoted $\phi$, we will instead use $l$. 

In [3]:
latlong.<𝛌,l> = S2.chart(r'𝛌:[-180,180] l:[-90,90]')

latlong.transition_map(
    spherical,
    [
        (-np.pi / 180.) * 𝛌,    # longitude to θ
        (-np.pi / 180.) * l     # latitude to φ
    ]
)


Change of coordinates from Chart (S^2_R, (𝛌, l)) to Chart (S^2_R, (θ, φ))

Then, we may proceed to define the locations of both Denver and London, as Points on the Manifold, in the geographic coordinate system.  Via the coordinate transition mappings defined above, we may then generate the corresponding points on the manifold $\mathcal{S}^2$ in our surface coordinate system $<\theta, \phi>$.

In [4]:
denver = S2.point( (-104.9847, 39.73915), chart=latlong)
print(denver)

denver.coord()


Point on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3


(1.83232867921849, -0.693579009443905)

In [5]:
london = S2.point( (-0.12574, 51.50853), chart=latlong)
print(london)

london.coord()


Point on the 2-sphere S^2_R of radius R smoothly embedded in the Euclidean space E^3


(0.00219457700145767, -0.898993441362275)

---

Here, we now need to solve the hom. Geodesic Equation for the minimal distance between these two points.  This equation serves as our "Equation of Motion (EoM)" between the two points, and is related to the Parallel Transport problem (and associated transport of the tangent vector space at every point along the geodesic curve between the two points).  Then, we may perform a contraction over the metric to arrive at a function for normed length (differential length along curve) in variables $\left(\theta, \phi\right)$, that we may numerically integrate over (in terms of the implicit variable $t$), to arrive at an expression for the unit distance length along this geodesic curve connecting the two points, parameterized by constant $R$ (the radius of our $n$-Sphere). 

In [19]:
########
## Apply the Physical Model and Calculate
##      aka; "Physical Analysis - Plug'n'Chug"
########

var('t')    # implicit variable `t` (for "time")

# construct a vector on the surface (in the tangent space),
# pointing from Denver to London in the basis coordinate frame
v = S2.tangent_space(denver)( 
    (
        london.coord() + (-1 * denver.coord())
    ), 
    name='v' 
)
v.display()


v = 0.00219457700145767 ∂/∂θ - 0.898993441362275 ∂/∂φ

---

In [8]:
########
##  Visualize and Elaborate
##      aka; "Plot and Explain"
########

## 3D plot of the 2-Sphere

E3 = S2.ambient()   # get the ambient space embedding of Euclidean R^3, for visualization
cartesian.<x,y,z> = E3.chart()

𝚽 = S2.diff_map(E3, 
                {
                    (spherical, cartesian): 
                    [
                        sin(θ)*cos(φ), 
                        sin(θ)*sin(φ), 
                        cos(θ)
                    ]
                })
graticule = spherical.plot(cartesian, mapping=𝚽, 
                           number_values=23, 
                           color='black', label_axes=False)

########



########

show(
    (
        (graticule + sphere(opacity=0.5))   # "Earth"
    ), 
    frame=False,
    online=True
)


Graphics3d Object